In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 11.0 MB/s 
     |████████████████████████████████| 182 kB 80.9 MB/s 
     |████████████████████████████████| 7.6 MB 8.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 11.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 132 kB 76.9 MB/s 
     |████████████████████████████████| 451 kB 61.0 MB/s 
     |████████████████████████████████| 212 kB 75.7 MB/s 
     |████████████████████████████████| 127 kB 61.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
with open("/content/MyDrive/MyDrive/MyDrive/Harry.txt", "r") as input:
    input_ = input.read().strip()

a = nltk.tokenize.sent_tokenize(input_)
# result = a.replace('\u200c', '')
l = []
for i in a:
  i = i.replace('\n', '')
  l.append(i)

In [ ]:
l

['Mr and Mrs Dursley, of number four, Privet Drive, were proud tosay that they were perfectly normal, thank you very much.',
 'Theywere the last people you’d expect to be involved in anythingstrange or mysterious, because they just didn’t hold with suchnonsense.',
 'Mr Dursley was the director of a firm called Grunnings, whichmade drills.',
 'He was a big, beefy man with hardly any neck,although he did have a very large moustache.',
 'Mrs Dursley wasthin and blonde and had nearly twice the usual amount of neck,which came in very useful as she spent so much of her time craningover garden fences, spying on the neighbours.',
 'The Dursleys had asmall son called Dudley and in their opinion there was no finerboy anywhere.',
 'The Dursleys had everything they wanted, but they also had asecret, and their greatest fear was that somebody would discoverit.',
 'They didn’t think they could bear it if anyone found out aboutthe Potters.',
 'Mrs Potter was Mrs Dursley’s sister, but they hadn’tmet fo

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoints = "facebook/bart-large-cnn"

tokenizer = BartTokenizer.from_pretrained(model_checkpoints)
model = BartForConditionalGeneration.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
max([len(tokenizer.tokenize(sentence)) for sentence in l])

In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in l:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(l) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

7

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1020, 1005, 1011, 1018, 971, 982, 812]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1022, 1007, 1013, 1020, 973, 984, 814]

In [ ]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
summary = []
for input in inputs:
  output = model.generate(**input)
  a = tokenizer.decode(*output, skip_special_tokens=True)
  summary.append(a)

In [ ]:
print(summary)

In [ ]:
from datasets import load_dataset,Dataset
import datasets

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
import evaluate
rouge_score = evaluate.load("rouge")

In [ ]:
dataset = load_dataset("kmfoda/booksum")

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/kmfoda___csv/kmfoda--booksum-de8f789b49c4161c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tt = dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [ ]:
max_input = 1024
max_target = 512
batch_size = 3

In [ ]:
tt["train"][1]

{'bid': 27681,
 'is_aggregate': False,
 'source': 'cliffnotes',
 'chapter_path': 'all_chapterized_books/27681-chapters/03.txt',
 'summary_path': 'finished_summaries/cliffnotes/The Last of the Mohicans/section_2_part_0.txt',
 'book_id': 'The Last of the Mohicans.chapter 3',
 'summary_id': 'chapter 3',
 'content': None,
 'summary': '{"name": "Chapter 3", "url": "https://web.archive.org/web/20201101053205/https://www.cliffsnotes.com/literature/l/the-last-of-the-mohicans/summary-and-analysis/chapter-3", "summary": "In another part of the forest by the river a few miles to the west, Hawkeye and Chingachgook appear to be waiting for someone as they talk with low voices. It is now afternoon. The Indian and the scout are attired according to their forest habits: Chingachgook with his semi-nude, war-painted body and scalping tuft of hair, his tomahawk, scalping knife, and short rifle; Hawkeye with his hunting shirt, skin cap, buckskin leggings, knife, pouch and horn, and long rifle. They discus

In [ ]:
# inputs = [chapter for chapter in dataset['chapter']]
dataset

DatasetDict({
    train: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 1431
    })
    validation: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 1484
    })
})

In [ ]:
import pandas as pd

In [ ]:
a = pd.DataFrame(dataset["train"])
b = pd.DataFrame(dataset["test"])
c = pd.DataFrame(dataset["validation"])

In [ ]:
a.drop(['bid','summary_id','content','summary_url','is_aggregate','source','chapter_path','summary_path','book_id','summary','chapter_length','summary_name','summary_analysis','summary_length','analysis_length'], inplace = True, axis = 1)
b.drop(['bid','summary_id','content','summary_url','is_aggregate','source','chapter_path','summary_path','book_id','summary','chapter_length','summary_name','summary_analysis','summary_length','analysis_length'], inplace = True, axis = 1)
c.drop(['bid','summary_id','content','summary_url','is_aggregate','source','chapter_path','summary_path','book_id','summary','chapter_length','summary_name','summary_analysis','summary_length','analysis_length'], inplace = True, axis = 1)

In [ ]:
raw_dataset = datasets.DatasetDict(
    {"train": Dataset.from_pandas(a),
     "validation": Dataset.from_pandas(c),
     "test": Dataset.from_pandas(b)
    })

In [ ]:
def preprocess_data(data_to_process):
  #get all the dialogues
  # inputs = [chapter for chapter in data_to_process['chapter']]
  #tokenize the dialogues
  model_inputs = tokenizer(data_to_process['chapter'],  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary_text'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['chapter', 'summary_text'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['chapter', 'summary_text'],
        num_rows: 1484
    })
    test: Dataset({
        features: ['chapter', 'summary_text'],
        num_rows: 1431
    })
})

In [ ]:
tokenize_data = raw_dataset.map(preprocess_data, batched = True, remove_columns=['chapter', 'summary_text'])

  0%|          | 0/10 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
model_name = model_checkpoints.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-to-HP",
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    push_to_hub=True
    )

trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=rouge_score
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/rhitabrat/bart-large-cnn-finetuned-to-HP into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9600
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 2400
  Number of trainable parameters = 406290432


Epoch,Training Loss,Validation Loss
1,1.908200,1.774265


Saving model checkpoint to bart-large-cnn-finetuned-to-HP/checkpoint-500
Configuration saved in bart-large-cnn-finetuned-to-HP/checkpoint-500/config.json
Model weights saved in bart-large-cnn-finetuned-to-HP/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bart-large-cnn-finetuned-to-HP/checkpoint-500/tokenizer_config.json
Special tokens file saved in bart-large-cnn-finetuned-to-HP/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bart-large-cnn-finetuned-to-HP/tokenizer_config.json
Special tokens file saved in bart-large-cnn-finetuned-to-HP/special_tokens_map.json
Saving model checkpoint to bart-large-cnn-finetuned-to-HP/checkpoint-1000
Configuration saved in bart-large-cnn-finetuned-to-HP/checkpoint-1000/config.json
Model weights saved in bart-large-cnn-finetuned-to-HP/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bart-large-cnn-finetuned-to-HP/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bart-large-cnn-finet

TrainOutput(global_step=2400, training_loss=1.9918807474772136, metrics={'train_runtime': 7304.7266, 'train_samples_per_second': 1.314, 'train_steps_per_second': 0.329, 'total_flos': 2.08042041802752e+16, 'train_loss': 1.9918807474772136, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to bart-large-cnn-finetuned-to-HP
Configuration saved in bart-large-cnn-finetuned-to-HP/config.json
Model weights saved in bart-large-cnn-finetuned-to-HP/pytorch_model.bin
tokenizer config file saved in bart-large-cnn-finetuned-to-HP/tokenizer_config.json
Special tokens file saved in bart-large-cnn-finetuned-to-HP/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.51G [00:00<?, ?B/s]

Upload file runs/Nov29_23-13-15_c1995d37f197/events.out.tfevents.1669763663.c1995d37f197.78.0:  56%|#####6    …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/rhitabrat/bart-large-cnn-finetuned-to-HP
   345b411..9f92366  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/rhitabrat/bart-large-cnn-finetuned-to-HP
   345b411..9f92366  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/rhitabrat/bart-large-cnn-finetuned-to-HP
   9f92366..76f9a2c  main -> main

   9f92366..76f9a2c  main -> main



'https://huggingface.co/rhitabrat/bart-large-cnn-finetuned-to-HP/commit/9f923660697beac00c65fb31e88e1c05f807fabf'

Using the fine-tuned model from the Hugging face

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model_checkpoints = "rhitabrat/bart-large-cnn-finetuned-to-HP"

tokenizer = BartTokenizer.from_pretrained(model_checkpoints)
model = BartForConditionalGeneration.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/999k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
max([len(tokenizer.tokenize(sentence)) for sentence in l])

103

In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in l:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(l) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

7

In [ ]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
summary = []
for input in inputs:
  output = model.generate(**input)
  a = tokenizer.decode(*output, skip_special_tokens=True)
  summary.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
summary

["Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense. The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn't think they could bear it if anyone found out about the Potters. Mrs. Potter is Mrs. and Dudley's sister and they hadn't met for several years, but Mrs. pretended she didn't have a sister because her sister and her",
 "Mr. Dursley has a normal, owl-free morning. He makes several important phone calls and is in a good mood until lunchtime, when he walks across the street to buy a bun from the baker's. As he passes a group of people in cloaks, he hears them talking about the Potters and their son, Harry. He stops dead. He looks back at them as if he wants to say something to them, but he t

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')


with open("/content/MyDrive/MyDrive/MyDrive/PS1_english.txt", "r") as input:
    input_ = input.read().strip()

a = nltk.tokenize.sent_tokenize(input_)
# result = a.replace('\u200c', '')
l = []
for i in a:
  i = i.replace('\n', '')
  l.append(i)

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in l:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(l) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

3

In [ ]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
summary1 = []
for input in inputs:
  output = model.generate(**input)
  a = tokenizer.decode(*output, skip_special_tokens=True)
  summary1.append(a)

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
summary1

['Vandiya Devan, a young warrior from the Vaanar clan, is riding down the banks of the Veera Narayana Lake, a huge reservoir in the southern end of Thirumunaipadi. The lake is about a league and a half long north to south and about half a league wide east to west. It is known as Veeraanatthu Lake, which means "an ocean-like reservoir" in Tamil. In the windy months of Aadi-Aavani, when new floods fill the reservoir to almost overflow, anyone who looks at the lake will surely recall with pride and astonishment the splendid deeds of our ancestors in Tamil',
 'Vandiya Devan arrives at the Aadi Festival, a festival celebrating the festival of the festival. People from the nearby villages have come from all over the country to celebrate the festival, and the festival is being celebrated by a large number of people. Aadi is a festival in which people gather together to sing and dance. The festival is also a time of celebration and celebration for the people of the area, and it is a time when 

In [ ]:
summary